In [1]:
import numpy as np

rn_20 = np.array([46, 102, 257, 406, 510, 630, 772, 880, 981, 1088, 1163, 1206, 1311, 1436, 1543, 1644, 1735, 1810, 1956, 2017])
rn_80 = np.array([87, 201, 286, 332, 449, 642, 675, 682, 744, 770, 883, 1013, 1085, 1227, 1237, 1298, 1310, 1333, 1358, 1671, 1728, 1812, 1819, 1963, 2295, 2410, 2450, 2508, 2553, 2592, 2700, 2976, 3084, 3359, 3557, 3780, 3879, 4303, 4328, 4397, 4428, 4514, 4561, 4852, 4897, 4943, 5000, 5187, 5314, 5361, 5532, 5535, 5635, 5646, 5718, 5853, 5898, 5917, 6056, 6164, 6340, 6435, 6827, 6839, 6852, 6944, 7174, 7466, 7494, 7496, 7497, 7519, 7524, 7604, 7638, 7788, 7830, 8152, 8158, 8173])

print (np.mean(rn_20))
print (np.mean(rn_80))


for i in range(10, 34, 4):
    test_indx = np.random.uniform(0,1,i)
    test_indx = np.sort(np.around(test_indx * 2048))
    test_indx = test_indx.astype(np.int)
    
    print(i, ":", test_indx)

for i in range(4, 8, 4):
    test_indx = np.random.uniform(0,1,i)
    test_indx = np.sort(np.around(test_indx * 2048))
    test_indx = test_indx.astype(np.int)
    
    print(i, ":", test_indx)






1074.65
4131.7125
10 : [  80  256  884 1013 1029 1117 1146 1455 1737 1923]
14 : [ 117  257  789  825  895  924  940  989 1045 1052 1281 1309 1541 2031]
18 : [  11   63  287  420  657  703  705  733  769  959 1340 1538 1586 1651
 1699 1942 1991 2027]
22 : [  67   94  103  130  390  448  662  679  745  819  978 1248 1293 1407
 1429 1635 1636 1658 1662 1894 1926 1947]
26 : [ 165  236  374  392  519  520  604  740  776  898  929 1174 1178 1216
 1465 1483 1532 1646 1692 1769 1793 1843 1847 1922 2001 2039]
30 : [  35  121  131  209  262  328  421  433  490  703  755  771  788  844
  873  983 1037 1068 1076 1088 1175 1411 1435 1458 1630 1678 1813 1871
 1892 1997]
4 : [  60  625 1131 1714]


In [8]:
import re
import csv
import glob
import numpy as np
from statistics import geometric_mean as gm



In [13]:
rx_dict = {
    'IPC': re.compile(r'CPU\s*0\s*cummulative\s*IPC\D*(?P<IPC>\S*)'),
    'llc_hit': re.compile(r'LLC.*TOTAL.*HIT\D*(?P<llc_hit>\S*)'),
    'llc_miss': re.compile(r'LLC.*TOTAL.*MISS\D*(?P<llc_miss>\S*)'),
    'load_hit': re.compile(r'LLC.*LOAD.*HIT\D*(?P<load_hit>\S*)'),
    'load_miss': re.compile(r'LLC.*LOAD.*MISS\D*(?P<load_miss>\S*)'),
    'rfo_hit': re.compile(r'LLC.*RFO.*HIT\D*(?P<rfo_hit>\S*)'),
    'rfo_miss': re.compile(r'LLC.*RFO.*MISS\D*(?P<rfo_miss>\S*)'),
    'prftch_hit': re.compile(r'LLC.*PREFETCH.*HIT\D*(?P<prftch_hit>\S*)'),
    'prftch_miss': re.compile(r'LLC.*PREFETCH.*MISS\D*(?P<prftch_miss>\S*)'),
    'wb_hit': re.compile(r'LLC.*WRITEBACK.*HIT\D*(?P<wb_hit>\S*)'),
    'wb_miss': re.compile(r'LLC.*WRITEBACK.*MISS\D*(?P<wb_miss>\S*)'),
}

#update this with the list of policies you wanna extract results of
model_list = ['lru-config2', 'lime-config2', 'lime_10ts-config2', 'lime_14ts-config2', 'lime_18ts-config2', 'lime_22ts-config2', 'lime_26ts-config2', 'lime_30ts-config2', 'lime_4ts-config2', 'lime_ht_16-config2', 'lime_ht_160-config2', 'lime_ht_192-config2', 'lime_ht_32-config2', 'lime_ht_64-config2', 'lime_ht_96-config2', 'lime_pat30-config2', 'lime_pc10-config2', 'lime_pc12-config2', 'lime_pc14-config2', 'lime_pc18-config2', 'lime_pc20-config2', 'lime_pc22-config2', 'multp-config2', 'red-config2', 'red_1_52-config2', 'red_259-config2', 'red_268-config2', 'red_2_60-config2', 'red_4_66-config2', 'red_5_69-config2', 'red_ad-config2', 'red_ah-config2', 'red_aq-config2', 'red_aqp-config2', 'red_ld-config2', 'red_lh-config2', 'red_lq-config2', 'red_lqp-config2', 'red_sw3-config2', 'red_sw4-config2', 'red_sw5-config2', 'ship_pp-config2']

all_result_dict = {}
trace_list= []

for model in model_list:
    file_list = glob.glob(f"results/*{model}-*")
    
    #print(file_list)

    for filepath in file_list:
        result_dict = {}
        trace_tag = re.match(r'.*-(\S*)\.txt',filepath).group(1)
        
        with open(filepath,'r') as file:
            line = file.readline()
            while line:
                line = file.readline()

                for key,rx in rx_dict.items():
                    match = rx.match(line)
                    if match: 
                        result_dict[key] = float(match.group(key))
        if  trace_tag not in all_result_dict.keys(): all_result_dict[trace_tag] = {}
        all_result_dict[trace_tag][model] = result_dict

trace_list = []
for trace in all_result_dict.keys():
    trace_list.append(trace)       

trace_list_len = len(trace_list)
model_list_len = len(model_list)

np_trace_list = np.array(trace_list)


np_trace_list = np.transpose(np_trace_list)
np_trace_list = np_trace_list.reshape((trace_list_len,1))


IPC_SPEED     = np.zeros((trace_list_len, model_list_len))
llc_hit       = np.zeros((trace_list_len, model_list_len))     
llc_miss      = np.zeros((trace_list_len, model_list_len))   
load_hit      = np.zeros((trace_list_len, model_list_len))   
load_miss     = np.zeros((trace_list_len, model_list_len))   
rfo_hit       = np.zeros((trace_list_len, model_list_len)) 
rfo_miss      = np.zeros((trace_list_len, model_list_len))   
prftch_hit    = np.zeros((trace_list_len, model_list_len))     
prftch_miss   = np.zeros((trace_list_len, model_list_len))     
wb_hit        = np.zeros((trace_list_len, model_list_len)) 
wb_miss       = np.zeros((trace_list_len, model_list_len)) 


for i in range(0,len(model_list)):
        model = model_list[i]
        for index, trace in enumerate(all_result_dict.keys()):
            #print(model, trace, all_result_dict[trace][model].keys())
            #print(trace, model, all_result_dict[trace][model]['IPC'])    
            IPC_SPEED[index][i]    = (all_result_dict[trace][model]['IPC']/all_result_dict[trace][model_list[0]]['IPC']) -1
            llc_hit[index][i]      = int(all_result_dict[trace][model]['llc_hit'])    
            llc_miss[index][i]     = int(all_result_dict[trace][model]['llc_miss'])   
            load_hit[index][i]     = int(all_result_dict[trace][model]['load_hit'])   
            load_miss[index][i]    = int(all_result_dict[trace][model]['load_miss'])  
            rfo_hit[index][i]      = int(all_result_dict[trace][model]['rfo_hit'])    
            rfo_miss[index][i]     = int(all_result_dict[trace][model]['rfo_miss'])   
            prftch_hit[index][i]   = int(all_result_dict[trace][model]['prftch_hit']) 
            prftch_miss[index][i]  = int(all_result_dict[trace][model]['prftch_miss'])
            wb_hit[index][i]       = int(all_result_dict[trace][model]['wb_hit'])     
            wb_miss[index][i]      = int(all_result_dict[trace][model]['wb_miss'])    

           

IPC_SPEED_w_trace   = np.concatenate((np_trace_list, IPC_SPEED) ,axis = 1)
llc_hit_w_trace     = np.concatenate((np_trace_list, llc_hit) ,axis = 1)    
llc_miss_w_trace    = np.concatenate((np_trace_list, llc_miss) ,axis = 1)   
load_hit_w_trace    = np.concatenate((np_trace_list, load_hit) ,axis = 1)   
load_miss_w_trace   = np.concatenate((np_trace_list, load_miss) ,axis = 1)  
rfo_hit_w_trace     = np.concatenate((np_trace_list, rfo_hit) ,axis = 1)    
rfo_miss_w_trace    = np.concatenate((np_trace_list, rfo_miss) ,axis = 1)   
prftch_hit_w_trace  = np.concatenate((np_trace_list, prftch_hit) ,axis = 1) 
prftch_miss_w_trace = np.concatenate((np_trace_list, prftch_miss) ,axis = 1)
wb_hit_w_trace      = np.concatenate((np_trace_list, wb_hit) ,axis = 1)     
wb_miss_w_trace     = np.concatenate((np_trace_list, wb_miss) ,axis = 1)

# print(IPC_SPEED_w_trace)
# print(llc_hit_w_trace)     
# print(llc_miss_w_trace)    
# print(load_hit_w_trace)    
# print(load_miss_w_trace)   
# print(rfo_hit_w_trace)     
# print(rfo_miss_w_trace)    
# print(prftch_hit_w_trace)  
# print(prftch_miss_w_trace) 
# print(wb_hit_w_trace)      
# print(wb_miss_w_trace)


In [14]:
# IPC_SPEED_w_trace  
# llc_hit_w_trace    
# llc_miss_w_trace   
# load_hit_w_trace   
# load_miss_w_trace  
# rfo_hit_w_trace    
# rfo_miss_w_trace   
# prftch_hit_w_trace 
# prftch_miss_w_trace
# wb_hit_w_trace     
# wb_miss_w_trace    



filename = "IPC.csv"
with open(filename, 'w') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile) 
        
    # writing the fields 
    csvwriter.writerow(model_list) 
        
    # writing the data rows 
    csvwriter.writerows(IPC_SPEED_w_trace)

filename = "llc_hit.csv"
with open(filename, 'w') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile) 
        
    # writing the fields 
    csvwriter.writerow(model_list) 
        
    # writing the data rows 
    csvwriter.writerows(llc_hit_w_trace)
    
filename = "llc_miss.csv"
with open(filename, 'w') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile) 
        
    # writing the fields 
    csvwriter.writerow(model_list) 
        
    # writing the data rows 
    csvwriter.writerows(llc_miss_w_trace)
    
filename = "load_hit.csv"
with open(filename, 'w') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile) 
        
    # writing the fields 
    csvwriter.writerow(model_list) 
        
    # writing the data rows 
    csvwriter.writerows(load_hit_w_trace)
    
filename = "load_miss.csv"
with open(filename, 'w') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile) 
        
    # writing the fields 
    csvwriter.writerow(model_list) 
        
    # writing the data rows 
    csvwriter.writerows(load_miss_w_trace)
    
filename = "rfo_hit.csv"
with open(filename, 'w') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile) 
        
    # writing the fields 
    csvwriter.writerow(model_list) 
        
    # writing the data rows 
    csvwriter.writerows(rfo_hit_w_trace)
    
filename = "rfo_miss.csv"
with open(filename, 'w') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile) 
        
    # writing the fields 
    csvwriter.writerow(model_list) 
        
    # writing the data rows 
    csvwriter.writerows(rfo_miss_w_trace)
    
filename = "prftch_hit.csv"
with open(filename, 'w') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile) 
        
    # writing the fields 
    csvwriter.writerow(model_list) 
        
    # writing the data rows 
    csvwriter.writerows(prftch_hit_w_trace)

filename = "prftch_miss.csv"
with open(filename, 'w') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile) 
        
    # writing the fields 
    csvwriter.writerow(model_list) 
        
    # writing the data rows 
    csvwriter.writerows(prftch_miss_w_trace)

filename = "wb_hit.csv"
with open(filename, 'w') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile) 
        
    # writing the fields 
    csvwriter.writerow(model_list) 
        
    # writing the data rows 
    csvwriter.writerows(wb_hit_w_trace)

filename = "wb_miss.csv"
with open(filename, 'w') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile) 
        
    # writing the fields 
    csvwriter.writerow(model_list) 
        
    # writing the data rows 
    csvwriter.writerows(wb_miss_w_trace)
